In [2]:
# For Database connection
from sqlalchemy import create_engine
# For NLP related tasks
import re
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
# For data modelling and evaluating pipeline
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, f1_score, make_scorer
from sklearn.model_selection import train_test_split
# Saving the model
import pickle

[nltk_data] Downloading package punkt to /Users/tugberk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tugberk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tugberk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tugberk/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
from models.train_classifier import load_data, tokenize, build_model, evaluate_model

[nltk_data] Downloading package punkt to /Users/tugberk/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tugberk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tugberk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/tugberk/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
X, y, category_names = load_data("data/DisasterResponse.db", "PostEtl")
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

In [4]:
category_names

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [17]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn import pipeline
from imblearn.over_sampling import SMOTE

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)


In [7]:
X_test

array(['AP 1:9-10/GAL 4:10-11/COL 2:14-18 ',
       "i would let every body know that the 4636 service for the emergency's information that i can't understand it ",
       'i need water, 27 bertin street, Carrefour', ...,
       'Due to the delays in obtaining clearance for the necessary cold chain logistics and vaccines for the immunisation component, the teams had to postpone their departure to the agreed project sites.',
       'The people in Fontamara are starving. We are precisely in Trou Sable Madame Ganot. Please pass on the message',
       'My wife and teenage children are not skilled in any particular trade but we will do whatever we can to help . My 13-year-old can take care of pets . My wife and 16-year-old daughter can help with child care . I can help with debris removal or other such tasks , such as delivering food/water to the elderly . ( My 86-year-old mother-in-law lives in a high rise on Pitt Street . )'],
      dtype=object)

In [22]:
pipeline = Pipeline([
   ('vectorizer', CountVectorizer(tokenizer=tokenize)),
   ('tfidf', TfidfTransformer()),
   ('classifier',MultiOutputClassifier(CatBoostClassifier(iterations=10,
                                                          verbose=True)))
])
# hyper-parameter grid
parameters = {'classifier__estimator__depth': [3,4,5],
              'classifier__estimator__learning_rate':[0.1, 0.3]}

# create model
model = GridSearchCV(estimator=pipeline,
                     param_grid=parameters,
                     verbose=3,
                     cv=3)
model.fit(X_train, y_train)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SMOTE()' (type <class 'imblearn.over_sampling._smote.base.SMOTE'>) doesn't

In [13]:
y_pred = model.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred, target_names=category_names))

                        precision    recall  f1-score   support

               related       0.78      1.00      0.87      4041
               request       0.81      0.35      0.49       948
                 offer       0.00      0.00      0.00        25
           aid_related       0.70      0.51      0.59      2198
          medical_help       0.71      0.01      0.02       400
      medical_products       0.86      0.02      0.04       268
     search_and_rescue       0.33      0.01      0.03       149
              security       0.00      0.00      0.00        83
              military       0.00      0.00      0.00       167
                 water       0.68      0.65      0.66       334
                  food       0.75      0.77      0.76       587
               shelter       0.83      0.47      0.60       471
              clothing       0.78      0.26      0.39        81
                 money       0.00      0.00      0.00       131
        missing_people       0.55      

/Users/tugberk/Downloads/disaster_response_pipeline_project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tugberk/Downloads/disaster_response_pipeline_project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/tugberk/Downloads/disaster_response_pipeline_project/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` para

In [15]:
model.best_params_

{'classifier__estimator__depth': 5,
 'classifier__estimator__learning_rate': 0.1}

In [32]:
for i in  range(len(y_train.columns)):
   y_train[y_train.columns[i]] = y_train[y_train.columns[i]].map(lambda x : 1 if x == 2 else x)


In [35]:
Y.related.max()

2

In [13]:
for i in range(4, df.shape[1], 1):
   print(df.columns[i])

related
request
offer
aid_related
medical_help
medical_products
search_and_rescue
security
military
child_alone
water
food
shelter
clothing
money
missing_people
refugees
death
other_aid
infrastructure_related
transport
buildings
electricity
tools
hospitals
shops
aid_centers
other_infrastructure
weather_related
floods
storm
fire
earthquake
cold
other_weather
direct_report


In [10]:
df.shape[1]

40

In [6]:
import pandas as pd
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('PostEtl', engine)
print(df.head())
# load model
#model = joblib.load("../models/CatClassifier.pkl")
#print("model_loaded")

   id                                            message  \
0   2  Weather update - a cold front from Cuba that c...   
1   7            Is the Hurricane over or is it not over   
2   8                    Looking for someone but no name   
3   9  UN reports Leogane 80-90 destroyed. Only Hospi...   
4  12  says: west side of Haiti, rest of the country ...   

                                            original   genre  related  \
0  Un front froid se retrouve sur Cuba ce matin. ...  direct        1   
1                 Cyclone nan fini osinon li pa fini  direct        1   
2  Patnm, di Maryani relem pou li banm nouvel li ...  direct        1   
3  UN reports Leogane 80-90 destroyed. Only Hospi...  direct        1   
4  facade ouest d Haiti et le reste du pays aujou...  direct        1   

   request  offer  aid_related  medical_help  medical_products  ...  \
0        0      0            0             0                 0  ...   
1        0      0            1             0              

In [12]:
genre_df = df.groupby('genre')['message'].count().reset_index()
#genre_names = list(genre_counts.index.str.replace("_", " ").str.title())

In [14]:
genre_df["genre"]

0    direct
1      news
2    social
Name: genre, dtype: object

In [17]:
genre_df["message"]

0    10766
1    13054
2     2396
Name: message, dtype: int64

In [18]:
genre_df

,genre,message
0,direct,10766
1,news,13054
2,social,2396


In [19]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df["len_msg"] = df.message.str.len()

In [28]:
df.groupby("genre")["len_msg"].agg(avg_len = "mean")

,avg_len
genre,
direct,89.952071
news,193.183852
social,127.186978


In [35]:
df.columns[:4]

Index(['id', 'message', 'original', 'genre'], dtype='object')

In [48]:
for i in range(4, df.shape[1], 1):
    #print(df.loc[df[df.columns[i]] == 1]["len_msg"].mean())
    print(df.columns[i])

related
request
offer
aid_related
medical_help
medical_products
search_and_rescue
security
military
child_alone
water
food
shelter
clothing
money
missing_people
refugees
death
other_aid
infrastructure_related
transport
buildings
electricity
tools
hospitals
shops
aid_centers
other_infrastructure
weather_related
floods
storm
fire
earthquake
cold
other_weather
direct_report
len_msg


In [45]:
df.loc[df.aid_related == 1]["len_msg"].mean()

173.26316758747697

In [60]:
col_names = []
len_message = []
for i in range(4, df.shape[1]-1, 1):
    len_message.append(df.loc[df[df.columns[i]] == 1]["len_msg"].mean())
    col_names.append(df.columns[i])

In [61]:
len(col_names)

36

In [62]:
len(len_message)

36